In [4]:
import csv
from googletrans import Translator
translator = Translator()

In [3]:
qas = []
with open('qa_en.csv') as f:
    reader = csv.reader(f, delimiter=';', quotechar='\"')
    for row in reader:
        id_, question, answer = row
        qas.append({
            'id': id_,
            'question': question,
            'answer': answer
        })

In [11]:
for i, row in enumerate(qas):
    print('Progress: ', round(i/ len(qas), 2), i, end='\r')
    answer = row['answer']
    try:
        t = translator.translate(answer, src='en', dest='pt')
        qas[i]['answer_pt'] = t.text
    except:
        print('Error')
        break

In [12]:
import json

json.dump(qas, open('covidqa.json', 'w'), separators=(',', ':'), sort_keys=True, indent=4)

In [19]:
with open('covidqa.csv', 'w') as f:
    writer = csv.writer(f, delimiter=';', quotechar='\"', quoting=csv.QUOTE_MINIMAL)
    for qa in qas:
        writer.writerow([qa['id'], qa['question'], qa['question_pt'], qa['answer'], qa['answer_pt']])

# Recuperação de Informação 

In [23]:
from gensim.summarization.bm25 import BM25

import nltk
import string

punctuation = string.punctuation
stopwords = nltk.corpus.stopwords.words('portuguese')

In [26]:
corpus = []
for qa in qas:
    question = qa['question_pt']
    tokens = [w.lower() for w in nltk.word_tokenize(question) if w.lower() not in stopwords and w.lower() not in punctuation]
    corpus.append(tokens)
    
retrieval = BM25(corpus)

In [31]:
dialogflow = []
with open('qa.csv') as f:
    reader = csv.reader(f, delimiter=';', quotechar='\"')
    for row in reader:
        id_, question, answer, = row
        dialogflow.append({
            'id': id_,
            'question': question,
            'answer': answer
        })

In [62]:
result = []
for i, intent in enumerate(dialogflow):
    if 'question.' in intent['id']:
        q = intent['question']
        tokens = [w.lower() for w in nltk.word_tokenize(q) if w.lower() not in stopwords and w.lower() not in punctuation]
        scores = retrieval.get_scores(tokens)
        candidates = sorted([(qas[i], s) for i, s in enumerate(scores)], key=lambda x: x[1], reverse=True)[:10]

        intent['candidates'] = [{ 'candidate': { 
            'question': row[0]['question_pt'], 'answer': row[0]['answer_pt']
            }, 
            'score': row[1] 
        } for row in candidates if row[1] > 0]
        result.append({
            'question': intent['question'],
            'answer': intent['answer'],
            'candidate': intent['candidates']
        })
   
json.dump(result, open('candidates.json', 'w'), separators=(',', ':'), sort_keys=True, indent=4)

In [60]:
result[20]

{'question': 'Como devemos lavar a roupa de uma pessoa com coronavírus?',
 'answer': 'As peças de roupa de pessoas com sintomas gripais deverão ser colocadas em um cesto exclusivo, que deve ser pego, idealmente com luvas, por outra pessoa e despejado na máquina de lavar. Após colocar as roupas na máquina, as mãos devem higienizadas com água e sabão ou álcool em gel. Proceder à lavagem normalmente, sem necessidade de água sanitária.',
 'candidate': [{'candidate': {'question': 'Quem está em risco quando os profissionais de saúde deixam de lavar as mãos?',
    'answer': 'profissional de saúde, mas também para suas famílias e comunidades em que vivem'},
   'score': 6.3941287875650135},
  {'candidate': {'question': 'Com o que lavar as mãos para proteger a própria saúde é consistente?',
    'answer': 'os direitos à dignidade e a participação plena nas decisões relacionadas à assistência em crises humanitárias.'},
   'score': 6.3941287875650135},
  {'candidate': {'question': 'Qual é o melhor 